In [1]:
import pandas as pd
import numpy as np

Train Data

In [2]:
train_data=pd.DataFrame(pd.read_csv('train.csv'))
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_data.shape

(891, 12)

Data Cleaning and Preparation

In [4]:
# Handle null values
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
# Store columns with 35% data missing
drop_columns=train_data.isnull().sum()[train_data.isnull().sum()>(35/100*train_data.shape[0])]
drop_columns

Cabin    687
dtype: int64

In [6]:
# We drop Cabin 
train_data.drop(drop_columns.index,axis=1,inplace=True)

# Fill remaining null values with its mean
train_data.fillna(train_data.mean(),inplace=True)
train_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

In [7]:
train_data['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [8]:
# Fill Embarked null values with most frequent S
train_data['Embarked'].fillna('S',inplace=True)
train_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [9]:
# Correlation values
train_data.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.033207,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.069809,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.331339,0.083081,0.018443,-0.549500
Age,0.033207,-0.069809,-0.331339,1.000000,-0.232625,-0.179191,0.091566
SibSp,-0.057527,-0.035322,0.083081,-0.232625,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.179191,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.091566,0.159651,0.216225,1.000000


In [10]:
# Combine Parch and SibSp into FamilySize 
train_data['FamilySize']=train_data['Parch']+train_data['SibSp']
train_data.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize
PassengerId,1.000000,-0.005007,-0.035144,0.033207,-0.057527,-0.001652,0.012658,-0.040143
Survived,-0.005007,1.000000,-0.338481,-0.069809,-0.035322,0.081629,0.257307,0.016639
Pclass,-0.035144,-0.338481,1.000000,-0.331339,0.083081,0.018443,-0.549500,0.065997
Age,0.033207,-0.069809,-0.331339,1.000000,-0.232625,-0.179191,0.091566,-0.248512
SibSp,-0.057527,-0.035322,0.083081,-0.232625,1.000000,0.414838,0.159651,0.890712
Parch,-0.001652,0.081629,0.018443,-0.179191,0.414838,1.000000,0.216225,0.783111
Fare,0.012658,0.257307,-0.549500,0.091566,0.159651,0.216225,1.000000,0.217138
FamilySize,-0.040143,0.016639,0.065997,-0.248512,0.890712,0.783111,0.217138,1.000000


In [11]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     891 non-null    object 
 11  FamilySize   891 non-null    int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 83.7+ KB


Exploratory Data Analysis

In [12]:
# If Alone affects survival rate     Result: Alone had less chances of survival
train_data['Alone']=[0 if train_data['FamilySize'][i]>0 else 1 for i in train_data.index]
train_data.groupby(['Alone'])['Survived'].mean()

Alone
0    0.505650
1    0.303538
Name: Survived, dtype: float64

In [13]:
# Check Alone passengers fare          Result: Not-Alone had higher chances of a high ticket price
train_data[['Alone','Fare']].corr()

,Alone,Fare
Alone,1.000000,-0.271832
Fare,-0.271832,1.000000


In [14]:
# If Sex affects survival rate, 1-female 0-male        Result: Females had higher chances of survival
train_data['Sex']=[0 if train_data['Sex'][i]=='male' else 1 for i in train_data.index]
train_data.groupby(['Sex'])['Survived'].mean()

Sex
0    0.188908
1    0.742038
Name: Survived, dtype: float64

In [15]:
# Alternative method to check % survival based on sex
female=train_data[train_data.Sex == 1]['Survived']
rate_female = sum(female)/len(female)

male=train_data[train_data.Sex == 0]['Survived']
rate_male = sum(male)/len(male)

print("Female % survived:", rate_female)
print("Male % survived:", rate_male)

Female % survived: 0.7420382165605095
Male % survived: 0.18890814558058924


In [16]:
# If Embarked origin affects survival rate               Result: Those boarded from Cherbourg survived more in proportion 
train_data.groupby(['Embarked'])['Survived'].mean()

Embarked
C    0.553571
Q    0.389610
S    0.339009
Name: Survived, dtype: float64

Test Data

In [17]:
test_data=pd.DataFrame(pd.read_csv('test.csv'))
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [18]:
# Handle null values
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [19]:
# Store columns with 35% data missing
drop_columns=test_data.isnull().sum()[test_data.isnull().sum()>(35/100*test_data.shape[0])]
drop_columns

Cabin    327
dtype: int64

In [20]:
# We drop Cabin (it is also dropped in train dataset)                                         
test_data.drop(drop_columns.index,axis=1,inplace=True)

# Fill remaining null values with its mean
test_data.fillna(test_data.mean(),inplace=True)
test_data.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [21]:
# Combine Parch and SibSp into FamilySize 
test_data['FamilySize']=test_data['Parch']+test_data['SibSp']
test_data.corr()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,FamilySize
PassengerId,1.000000,-0.026751,-0.030874,0.003818,0.043080,0.008209,0.030087
Pclass,-0.026751,1.000000,-0.440782,0.001087,0.018721,-0.576619,0.012736
Age,-0.030874,-0.440782,1.000000,-0.079535,-0.045073,0.326800,-0.076072
SibSp,0.003818,0.001087,-0.079535,1.000000,0.306895,0.171488,0.788610
Parch,0.043080,0.018721,-0.045073,0.306895,1.000000,0.230001,0.827242
Fare,0.008209,-0.576619,0.326800,0.171488,0.230001,1.000000,0.249832
FamilySize,0.030087,0.012736,-0.076072,0.788610,0.827242,0.249832,1.000000


In [22]:
# Change Sex as 1-female 0-male       
test_data['Sex']=[0 if test_data['Sex'][i]=='male' else 1 for i in test_data.index]

In [23]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    int64  
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Embarked     418 non-null    object 
 10  FamilySize   418 non-null    int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 36.0+ KB


Model Building

In [24]:
# Split datasets
from sklearn.ensemble import RandomForestClassifier
Y_train=train_data["Survived"]

features=["Pclass","Sex","Age","Fare","Parch","SibSp","FamilySize"]
X_train=pd.get_dummies(train_data[features])
X_test=pd.get_dummies(test_data[features])

In [25]:
# Fitting Random Forest Regression to the dataset
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

model_random_forest=RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=5, random_state=1)
model = Pipeline([('scaler2' , StandardScaler()),('RandomForestRegressor: ', model_random_forest)])
model.fit(X_train,Y_train)
predictions = model.predict(X_test)

In [26]:
# Check error in Training model
from sklearn.metrics import mean_squared_error

y_train_predict=model.predict(X_train)
rms_err=(np.sqrt(mean_squared_error(Y_train,y_train_predict)))
print("Train RMSE: {}".format(rms_err) )

Train RMSE: 0.3892494720807615


In [27]:
# Accuracy % for train set
model.score(X_train,Y_train)

0.8484848484848485

In [28]:
# Write output file as my_submission.csv
output=pd.DataFrame({'PassengerId':test_data.PassengerId,'Survived':predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
